### IFN680 Sokoban Assignment


The functions and classes defined in this module will be called by a marker script. 
You should complete the functions and classes according to their specified interfaces.

You are not allowed to change the defined interfaces.
That is, changing the formal parameters of a function will break the 
interface and triggers to a fail for the test of your code.
 

In [42]:
# Imports
import search
import sokoban
from sokoban import find_2D_iterator
import math
from collections import namedtuple
from sokoban import Warehouse

In [43]:
def my_team():
    '''
    Return the list of the team members of this assignment submission as a list
    of triplet of the form (student_number, first_name, last_name)
    
    '''
    return [(11771984, 'Daniel', 'Skauge'), (11771861, 'Erlend', 'Eiring')]

In [1]:
def taboo_cells(warehouse):
        '''  
    Identify the taboo cells of a warehouse. A cell inside a warehouse is 
    called 'taboo' if whenever a box get pushed on such a cell then the puzzle 
    becomes unsolvable.  
    When determining the taboo cells, you must ignore all the existing boxes, 
    simply consider the walls and the target cells.  
    Use only the following two rules to determine the taboo cells;
     Rule 1: if a cell is a corner inside the warehouse and not a target, 
             then it is a taboo cell.
     Rule 2: all the cells between two corners inside the w]arehouse along a 
             wall are taboo if none of these cells is a target.
    
    @param warehouse: a Warehouse object

    @return
       A string representing the puzzle with only the wall cells marked with 
       an '#' and the taboo cells marked with an 'X'.  
       The returned string should NOT have marks for the worker, the targets,
       and the boxes.  
    '''
        max_x, max_y = get_warehouse_dims(warehouse)

        warehouse_grid = [[' ' for _ in range(max_x + 1)] for _ in range(max_y + 1)]
        
        for (x,y) in warehouse.walls:
                warehouse_grid[y][x] = '#'


        for x in range(max_x+1):
                for y in range(max_y+1):
                        if is_corner(x,y,warehouse_grid) and (x,y) not in warehouse.targets:
                                warehouse_grid[y][x] = 'X'
                        elif is_between_corners_without_targets(x,y, warehouse.targets, warehouse_grid):
                                warehouse_grid[y][x] = 'X'

        return warehouse_grid_to_string(warehouse_grid)

def get_taboo_cells_positions(warehouse):
    taboo_cells_string = taboo_cells(warehouse)
    taboo_cells_lines = taboo_cells_string.split('\n')
    positions_generator = find_2D_iterator(taboo_cells_lines, 'X')
    return tuple(positions_generator)


     
def is_corner(x, y, warehouse_grid):
    if not is_within_grid(x, y, warehouse_grid):
        return False
    if warehouse_grid[y][x] == '#':
        return False

    top = is_within_grid(
        x, y-1, warehouse_grid) and warehouse_grid[y-1][x] == '#'
    bottom = is_within_grid(
        x, y+1, warehouse_grid) and warehouse_grid[y+1][x] == '#'
    left = is_within_grid(
        x-1, y, warehouse_grid) and warehouse_grid[y][x-1] == '#'
    right = is_within_grid(
        x+1, y, warehouse_grid) and warehouse_grid[y][x+1] == '#'

    return (top and left) or (top and right) or (bottom and left) or (bottom and right)


def is_between_corners_without_targets(x, y, targets, warehouse_grid):
    horizontal_taboo = vertical_taboo = False

    # Vertical checks
    if between_vertical_corners(x, y, warehouse_grid):
        y_range = range_between_corners(x, y, 'y', warehouse_grid)
        continuous_wall_left = is_continuous_wall_left(x, y, y_range, warehouse_grid)  
        continuous_wall_right = is_continuous_wall_right(x, y, y_range, warehouse_grid)
        no_targets_vertical = is_no_targets_vertical(x, y_range, targets)
        vertical_taboo = (continuous_wall_left or continuous_wall_right) and no_targets_vertical

    # Horizontal checks
    if between_horizontal_corners(x, y, warehouse_grid):
        x_range = range_between_corners(x, y, 'x', warehouse_grid)
        continuous_wall_top = is_continuous_wall_top(x,y, x_range, warehouse_grid)
        continuous_wall_bottom = is_continuous_wall_bottom(x,y, x_range, warehouse_grid)
        no_targets_horizontal = is_no_targets_horizontal(y, x_range, targets)
        horizontal_taboo = (continuous_wall_top or continuous_wall_bottom) and no_targets_horizontal

    return horizontal_taboo or vertical_taboo


def range_between_corners(x, y, axis, warehouse_grid):
    left_corner = None
    right_corner = None
    top_corner = None
    bottom_corner = None

    if axis == 'x':
        for dx in range(x, -1, -1):
            if not is_within_grid(dx, y, warehouse_grid):
                break
            if is_corner(dx, y, warehouse_grid):
                left_corner = dx
                break

        for dx in range(x, len(warehouse_grid[0])):
            if not is_within_grid(dx, y, warehouse_grid):
                break
            if is_corner(dx, y, warehouse_grid):
                right_corner = dx
                break

    elif axis == 'y':
        for dy in range(y, -1, -1):
            if not is_within_grid(x, dy, warehouse_grid):
                break
            if is_corner(x, dy, warehouse_grid):
                top_corner = dy
                break

        for dy in range(y, len(warehouse_grid)):
            if not is_within_grid(x, dy, warehouse_grid):
                break
            if is_corner(x, dy, warehouse_grid):
                bottom_corner = dy
                break

    if axis == 'x':
        if left_corner is None or right_corner is None:
            return None
        return range(left_corner, right_corner + 1)

    elif axis == 'y':
        if top_corner is None or bottom_corner is None:
            return None
        return range(top_corner, bottom_corner + 1)


def between_horizontal_corners(x, y, warehouse_grid):
    left_corner, right_corner = False, False
    for dx in range(x, -1, -1):
        if is_corner(dx, y, warehouse_grid):
            left_corner = True
            break
        if warehouse_grid[y][dx] == '#':
            break
    for dx in range(x, len(warehouse_grid[0])):
        if is_corner(dx, y, warehouse_grid):
            right_corner = True
            break
        if warehouse_grid[y][dx] == '#':
            break
    return left_corner and right_corner


def between_vertical_corners(x, y, warehouse_grid):
    top_corner, bottom_corner = False, False
    for dy in range(y, -1, -1):
        if is_corner(x, dy, warehouse_grid):
            top_corner = True
            break
        if warehouse_grid[dy][x] == '#':
            break
    for dy in range(y, len(warehouse_grid)):
        if is_corner(x, dy, warehouse_grid):
            bottom_corner = True
            break
        if warehouse_grid[dy][x] == '#':
            break
    return top_corner and bottom_corner


def is_continuous_wall_top(x,y, x_range, warehouse_grid):
    if not is_within_grid(x,y-1,warehouse_grid): 
        return False
    else:
        return all(warehouse_grid[y-1][dx] == '#' for dx in x_range)


def is_continuous_wall_bottom(x,y, x_range, warehouse_grid):
    if not is_within_grid(x,y+1,warehouse_grid): 
        return False
    else:
        return all(warehouse_grid[y+1][dx] == '#' for dx in x_range)


def is_continuous_wall_left(x,y,y_range, warehouse_grid):
    if not is_within_grid(x-1,y,warehouse_grid): 
        return False
    else:
        return all(warehouse_grid[dy][x-1] == '#' for dy in y_range)


def is_continuous_wall_right(x,y,y_range, warehouse_grid):
    if not is_within_grid(x+1,y,warehouse_grid): 
        return False
    else:
        return all(warehouse_grid[dy][x+1] == '#' for dy in y_range)


def is_no_targets_vertical(x,y_range, targets):
    return all((x, dy) not in targets for dy in y_range)


def is_no_targets_horizontal(y, x_range, targets):
    return all((dx, y) not in targets for dx in x_range)


def get_warehouse_dims(warehouse):
    max_x = max(wall[0] for wall in warehouse.walls)
    max_y = max(wall[1] for wall in warehouse.walls)
    return max_x, max_y


def warehouse_grid_to_string(warehouse_grid):
    row_strings = [''.join(row) for row in warehouse_grid]
    warehouse_string = '\n'.join(row_strings)
    return warehouse_string


def is_within_grid(x, y, warehouse_grid):
    return 0 <= x < len(warehouse_grid[0]) and 0 <= y < len(warehouse_grid)


In [45]:
class SokobanPuzzle(search.Problem):
    '''
    An instance of the class 'SokobanPuzzle' represents a Sokoban puzzle.
    An instance contains information about the walls, the targets, the boxes
    and the worker.

    Your implementation should be fully compatible with the search functions of 
    the provided module 'search.py'. 
    
    Each instance should have at least the following attributes
    - self.allow_taboo_push
    - self.macro
    
    When self.allow_taboo_push is set to True, the 'actions' function should 
    return all possible legal moves including those that move a box on a taboo 
    cell. If self.allow_taboo_push is set to False, those moves should not be
    included in the returned list of actions.
    
    If self.macro is set True, the 'actions' function should return 
    macro actions. If self.macro is set False, the 'actions' function should 
    return elementary actions.
    
    
    '''
    def __init__(self, warehouse,allow_taboo_push=True,macro=True):
        State = namedtuple('State', ['walls', 'boxes', 'targets','worker'])
        self.state = State(
            walls=tuple(warehouse.walls),
            boxes=tuple(warehouse.boxes),
            targets=tuple(warehouse.targets),
            worker=tuple(warehouse.worker)
        )

        super().__init__(initial=self.state, goal=tuple(warehouse.targets))
        self.allow_taboo_push = allow_taboo_push
        self.taboo_cells = get_taboo_cells_positions(warehouse)
        self.macro = macro
        self.warehouse = warehouse

    def goal_test(self, state):
        return set(state.boxes) == set(self.goal)


    def actions(self, state):
        """
        Return the list of actions that can be executed in the given state.
        
        As specified in the header comment of this class, the attributes
        'self.allow_taboo_push' and 'self.macro' should be tested to determine
        what type of list of actions is to be returned.

        if elem
            check if the positions at each direction from the worker is wall or box 
            if box check if the cell one step beyond that box in the same dir is taboo,
              wall or other box, and determine if possible on that
            
        elif macro
            for each box, which direction can it move in based on next by taboo cells, wall or other box
                for each viable direction, can the worker reach the cell it needs to be in to move the box without moving other boxes on the way
                    add each viable direction to the action set
        """
        if self.macro:
            return self.macro_actions(state)
        else:
            return self.elementary_actions(state)
        
    def macro_actions(self, state):
        actions = []
        directions = ['left', 'up', 'right', 'down']
        for box in state.boxes:
            for dir in directions:
                if self.is_valid_macro_action(box, dir, state):
                    actions.append((box, dir))
        return actions

    def is_valid_macro_action(self, box, dir, state):
        candidate_cell = self.get_neighbor_cell_in_direction(box, dir)
        if not self.warehouse.is_in_warehouse(candidate_cell):
            return False
        if candidate_cell in state.walls or candidate_cell in state.boxes:
            return False
        if candidate_cell in self.taboo_cells and not self.allow_taboo_push:
            return False
        worker_push_position = self.get_worker_push_position(box, dir)
        return can_go_there(self.warehouse, worker_push_position)

    def elementary_actions(self, state):
        actions = []
        directions = ['left', 'up', 'right', 'down']
        for dir in directions:
            if self.is_valid_elementary_action(state.worker, dir, state):
                actions.append(dir)
        return actions

    def is_valid_elementary_action(self, worker, dir, state):
        candidate_cell = self.get_neighbor_cell_in_direction(worker, dir)
        if not self.warehouse.is_in_warehouse(candidate_cell):
            return False
        if candidate_cell in state.walls:
            return False
        if candidate_cell in state.boxes:
            beyond_box_pos = self.get_neighbor_cell_in_direction(candidate_cell, dir)
            # print("Hittin the box man! l:112 soobanpuzzle")
            return self.is_push_possible(beyond_box_pos, state) # the move is valid, put the fact that boxes gets hit is overlooked?
        return True

    def is_push_possible(self, beyond_box_pos, state):
        return not (beyond_box_pos in state.boxes or beyond_box_pos in state.walls or
                    (beyond_box_pos in self.taboo_cells and not self.allow_taboo_push))
    
    def get_neighbor_cell_in_direction(self,cell,dir):
        return {
                'left': (cell[0]-1,cell[1]),
                'right': (cell[0]+1,cell[1]),
                'up': (cell[0],cell[1]-1),
                'down': (cell[0],cell[1]+1)
            }.get(dir)
    
    def get_worker_push_position(self, box,dir):
        return {
            'left': (box[0]+1,box[1]),
            'right': (box[0]-1,box[1]),
            'up': (box[0],box[1]+1),
            'down': (box[0],box[1]-1)
        }.get(dir)
    
    def result(self, state, action):
        return self.macro_result(state, action) if self.macro else self.elem_result(state, action)

    def macro_result(self, state, action):
        box_to_move_coords, direction = action
        new_boxes_coords = self.update_boxes_coords(state, box_to_move_coords, direction)
        new_worker_coords = box_to_move_coords
        return state._replace(boxes=new_boxes_coords, worker=new_worker_coords)

    def elem_result(self, state, action):
        direction = action
        new_worker_coords = self.update_worker_coords(state, direction)

        if new_worker_coords in state.boxes:
            new_boxes_coords = self.update_boxes_coords(state, new_worker_coords, direction)
        else:
            new_boxes_coords = state.boxes

        return state._replace(boxes=new_boxes_coords, worker=new_worker_coords)

    def update_boxes_coords(self, state, old_box_coords, direction):
        new_box_coords = self.get_neighbor_cell_in_direction(old_box_coords, direction)
        box_index = state.boxes.index(old_box_coords)
        return tuple(new_box_coords if i == box_index else box for i, box in enumerate(state.boxes))

    def update_worker_coords(self, state, direction):
        return self.get_neighbor_cell_in_direction(state.worker, direction)
    
    def manhattan_distance(self,coord1, coord2):
        """
        Calculate the Manhattan distance between two coordinates.
        
        Parameters:
        - coord1: tuple, (x1, y1)
        - coord2: tuple, (x2, y2)
        
        Returns:
        - int: Manhattan distance
        """
        return abs(coord1[0] - coord2[0]) + abs(coord1[1] - coord2[1])
    
    def h(self, node):
        """
        Calculate the generalized Manhattan distance as the heuristic function h.
        
        Parameters:
        - boxes: tuple of tuples, each containing (x, y) coordinates for boxes
        - targets: tuple of tuples, each containing (x, y) coordinates for targets
        
        Returns:
        - int: Generalized Manhattan distance
        """
        boxes = node.state.boxes
        targets = node.state.targets

        total_distance = 0
        
        for box in boxes:
            min_distance_to_any_target = float('inf')
            for target in targets:
                distance = self.manhattan_distance(box, target)
                min_distance_to_any_target = min(min_distance_to_any_target, distance)
            total_distance += min_distance_to_any_target
        
        return total_distance
    
    def euclidean_distance(self, point1, point2):
        """
        Calculate the Euclidean distance between two points.

        Parameters:
        - point1: Tuple (x1, y1) representing the coordinates of the first point
        - point2: Tuple (x2, y2) representing the coordinates of the second point

        Returns:
        - float: Euclidean distance between the two points
        """
        x1, y1 = point1
        x2, y2 = point2
        return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

    def h_euclid(self, node):
        """
        Calculate the generalized Euclidean distance as the heuristic function h.

        Parameters:
        - node: Current search node

        Returns:
        - float: Generalized Euclidean distance
        """
        boxes = node.state.boxes
        targets = node.state.targets

        total_distance = 0

        for box in boxes:
            min_distance_to_any_target = float('inf')
            for target in targets:
                distance = self.euclidean_distance(box, target)
                min_distance_to_any_target = min(min_distance_to_any_target, distance)
            total_distance += min_distance_to_any_target

        return total_distance

    


In [46]:
def check_action_seq(warehouse, action_seq):

    '''
    
    Determine if the sequence of actions listed in 'action_seq' is legal or not.
    
    Important notes:
      - a legal sequence of actions does not necessarily solve the puzzle.
      - an action is legal even if it pushes a box onto a taboo cell.
        
    @param warehouse: a valid Warehouse object

    @param action_seq: a sequence of legal actions.
           For example, ['Left', 'Down', Down','Right', 'Up', 'Down']
           
    @return
        The string 'Failure', if one of the action was not successul.
           For example, if the agent tries to push two boxes at the same time,
                        or push one box into a wall.
        Otherwise, if all actions were successful, return                 
               A string representing the state of the puzzle after applying
               the sequence of actions.  This must be the same string as the
               string returned by the method  Warehouse.__str__()
    '''
    
    ##         "INSERT YOUR CODE HERE"
    
    puzzel = SokobanPuzzle(warehouse, allow_taboo_push=True, macro=False)
    for action in action_seq:
        action = action.lower()
        if puzzel.is_valid_elementary_action(
            puzzel.state.worker,
            action,
            puzzel.state
        ):
          puzzel.state = puzzel.result(puzzel.state, action)
          warehouse = warehouse.copy(worker=puzzel.state.worker, 
                                     boxes=puzzel.state.boxes)
        else: 
           return 'Failure'
    
    
    return warehouse.__str__()


    

In [47]:
def solve_sokoban_elem(warehouse):
    #
    '''    
    This function should solve using elementary actions 
    the puzzle defined in a file.
    
    @param warehouse: a valid Warehouse object

    @return
        If puzzle cannot be solved return the string 'Impossible'
        If a solution was found, return a list of elementary actions that solves
            the given puzzle coded with 'Left', 'Right', 'Up', 'Down'
            For example, ['Left', 'Down', Down','Right', 'Up', 'Down']
            If the puzzle is already in a goal state, simply return []
    '''

    problem = SokobanPuzzle(warehouse,
                            allow_taboo_push=False,
                            macro=False)

    if problem.goal_test(problem.initial):
        return []

    solution_actions_sequence = search.breadth_first_graph_search(problem)

    if solution_actions_sequence is not None:
        return solution_actions_sequence

    return ['Impossible'] # skill issue..

In [48]:
def can_go_there(warehouse, dst):
    '''
    Determine whether the worker can walk to the cell dst=(row,column) 
    without pushing any box.
    
    @param warehouse: a valid Warehouse object

    @return
      True if the worker can walk to cell dst=(row,column) without pushing any box
      False otherwise
    '''
    if not warehouse.is_in_warehouse(dst):
        return False
    sub_problem = can_go_there(warehouse, dst)
    goal_node = search.astar_graph_search(
        sub_problem, sub_problem.manhattan_distance_heuristic)
    return goal_node is not None  # If it's None, we couldn't reach the position


In [49]:
def solve_sokoban_macro(warehouse, search_algorithm=search.breadth_first_graph_search, f_func=None, g_func=None):
    '''
    @param warehouse: a valid Warehouse object
    @param search_algorithm: a search function to use (e.g., breadth_first_graph_search, astar_search, etc.)
    @param f_func: heuristic function for f (only used for greedy and informed searches)
    @param g_func: cost function for g (only used for informed searches like A*)

    @return
        If puzzle cannot be solved return the string 'Impossible'
        Otherwise return M a sequence of macro actions that solves the puzzle.
        If the puzzle is already in a goal state, simply return []
    '''
    problem = SokobanPuzzle(warehouse, 
                            allow_taboo_push=False, 
                            macro=True)

    if problem.goal_test(problem.initial): 
        return []

    # Depending on the search algorithm, provide appropriate arguments
    solution_actions_sequence = search_algorithm(problem)

    if solution_actions_sequence:
        return solution_actions_sequence.solution()
    
    return 'Impossible'

